## Projeto de Webscrapping

#### Objetivo:
- Extrair os pedidos do site do fornecedor
- Inserir em outra plataforma online para bonificação de premios

#### Caminho a ser seguido:
- Abrir site de pedidos Zeiss
- Pegar os pedidos do mês anterior
- Inserir pedidos na plataforma de prêmios

In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

In [56]:
# Definição de variáveis e inserção de login e senhas que serão utilizadas

usuario = 'usuario_censurado'
senha = 'senha_censurada'
cnpj = 'cnpj_censurado'

pedidos_zeiss_realizados = []

hoje = datetime.today()
data_ajustada = hoje.replace(day=1) - timedelta(days=1)
ano = data_ajustada.year
ano = str(ano)[-2:]
mes_anterior = data_ajustada.month
if len(str(mes_anterior)) == 1:
    mes_anterior = '0' + str(mes_anterior)

In [57]:
# Realizando login no site do fornecedor

nav3 = webdriver.Chrome()
nav3.get('https://pedidolab.lenteszeiss.com.br/scripts/cgiip.exe/WService%3Dproducao/index.html')
nav3.switch_to.frame('conteudo')
nav3.find_element('xpath', '/html/body/div/div[1]/form/table/tbody/tr[2]/td[2]/input').send_keys('login_censurado')
nav3.find_element('xpath', '/html/body/div/div[1]/form/table/tbody/tr[3]/td[2]/input').send_keys('senha_censurada')
nav3.find_element('xpath', '/html/body/div/div[1]/form/table/tbody/tr[3]/td[2]/input').send_keys(Keys.ENTER)

main_window = nav3.current_window_handle

# Fechando Pop ups e voltando para a janela principal
for i in nav3.window_handles:
    if i != main_window:
        nav3.switch_to.window(i)
        nav3.close()

nav3.switch_to.window(main_window)

In [58]:
# Filtrando datas para extração dos números dos pedidos (obs: o site somente suporta pesquisa de no máximo 30 dias)

nav3.get('https://pedidolab.lenteszeiss.com.br/scripts/cgiip.exe/WService%3Dproducao/saow102.html?pAuto=NO')
nav3.find_element('id', 'txt_inicio').send_keys(Keys.CONTROL, 'a')
nav3.find_element('id', 'txt_inicio').send_keys('01/'+ str(mes_anterior) +f'/{ano}')
nav3.find_element('id', 'txt_fim').send_keys(Keys.CONTROL, 'a')

if mes_anterior == 2:
    nav3.find_element('id', 'txt_fim').send_keys('28/'+ str(mes_anterior) +f'/{ano}')
else:
    nav3.find_element('id', 'txt_fim').send_keys('30/'+ str(mes_anterior) +f'/{ano}')

nav3.find_element('id', 'b_enviar').click()

# Pegar todos os pedidos dentro da data selecionada.
while True:
    try:
        for i in range(10):
            pedidos_zeiss_realizados.append(nav3.find_element('xpath', '/html/body/form/table[3]/tbody/tr/td/table/tbody/tr['+ str(i+2) +']/td[3]').get_attribute('innerText'))
        nav3.find_element('xpath', '/html/body/form/p/table/tbody/tr/td[6]/a').click()
    except:
        break

# Em caso de meses com mais de 30 dias, o sistema faz uma nova consulta, pegando apenas o último dia faltante (31)
if mes_anterior in [1, 3, 5, 7, 8, 12]:
    nav3.get('https://pedidolab.lenteszeiss.com.br/scripts/cgiip.exe/WService%3Dproducao/saow102.html?pAuto=NO')
    nav3.find_element('id', 'txt_inicio').send_keys(Keys.CONTROL, 'a')
    nav3.find_element('id', 'txt_inicio').send_keys('31/'+ str(mes_anterior) +'/24')
    nav3.find_element('id', 'txt_fim').send_keys(Keys.CONTROL, 'a')
    nav3.find_element('id', 'txt_fim').send_keys('31/'+ str(mes_anterior) +'/24')
    nav3.find_element('id', 'b_enviar').click()

    while True:
        try:
            for i in range(10):
                pedidos_zeiss_realizados.append(nav3.find_element('xpath', '/html/body/form/table[3]/tbody/tr/td/table/tbody/tr['+ str(i+2) +']/td[3]').get_attribute('innerText'))
            nav3.find_element('xpath', '/html/body/form/p/table/tbody/tr/td[6]/a').click()
        except:
            break

In [60]:
# Abrindo site de premiação para inserção de pedidos armazenados

nav3.get('https://zeissone.com.br/')
nav3.find_element('id', 'Usuario').send_keys(usuario)
nav3.find_element('id', 'password-field').send_keys(senha)
nav3.find_element('id', 'password-field').send_keys(Keys.ENTER)
nav3.get('https://zeissone.com.br/Insercao')

# Inserção de pedidos
# Pedidos com premiação gerada com sucesso mostrarão uma mensagem com o produto e a pontuação liberada

for pedido in pedidos_zeiss_realizados:
    nav3.find_element('id', 'CNPJ').send_keys(Keys.CONTROL, 'a')
    nav3.find_element('id', 'CNPJ').send_keys(cnpj)
    nav3.find_element('id', 'Pedido').send_keys(Keys.CONTROL, 'a')
    nav3.find_element('id', 'Pedido').send_keys(pedido)
    nav3.execute_script("arguments[0].scrollIntoView();", nav3.find_element('id', 'Pedido'))
    time.sleep(0.3)
    nav3.find_element('id', 'btnEnviarCertificado').click()
    if 'sucesso' in nav3.find_element('xpath', '/html/body/div[2]/div/p').get_attribute('innerText'):
        print(nav3.find_element('xpath', '/html/body/div[2]/div/p').get_attribute('innerText'))


Produto VS ZEISS LIGHT 2 1.67 BLUEGUARD: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 1,800.00
Produto VS ZEISS MyoCare S 1.59: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 1,800.00
Produto ZEISS Acabadas ClearView Chrome 1.50: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 00.00
Produto ZEISS Acabadas ClearView Silver 1.67: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 00.00
Produto ZEISS Acabadas ClearView Chrome 1.50: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 00.00
Produto VS ZEISS LIGHT 2 1.50 BLUEGUARD: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 600.00
Produto VS ZEISS LIGHT 2 1.50 BLUEGUARD: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 600.00
Produto ZEISS Acabadas ClearView Silver 1.50: Seu certificado foi inserido com sucesso. Total de pontos liberado(s): 00.00
Produto ZEISS Acabadas ClearView Sil